In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.tokenize import word_tokenize
import os
import pickle
import string
from nltk.corpus import stopwords

In [2]:
class UnigramInvertedIndex:
    def __init__(self):
        pass

    def build_inverted_index(self, list_of_file_paths):
        index = {}
        for file_path in list_of_file_paths:
            with open(file_path, 'r') as f:
                line = f.readlines()[0]
                tokens = word_tokenize(line)
                for token in tokens:
                    if token not in index:
                        index[token] = [file_path.split('\\')[-1]]
                    else:
                        if file_path.split('\\')[-1] not in index[token]:
                            index[token].append(file_path.split('\\')[-1])
        self.index = index
        self.list_of_file_paths = list_of_file_paths
        self.file_names = [file_path.split('\\')[-1] for file_path in list_of_file_paths]

    def save(self, file_path):
        if self.index is None or self.list_of_file_paths is None or self.file_names is None:
            raise Exception('Index is not built yet')
        with open(file_path, 'wb') as f:
            pickle.dump([self.index, self.list_of_file_paths, self.file_names], f)

    def load(self, file_path):
        with open(file_path, 'rb') as f:
            self.index, self.list_of_file_paths, self.file_names = pickle.load(f)


In [3]:
files = os.listdir('..\data\preprocessed_data')
file_paths = []
for file in files:
    file_paths.append('..\data\preprocessed_data\\' + file)

In [4]:
unigram_inverted_index = UnigramInvertedIndex()

In [5]:
unigram_inverted_index.build_inverted_index(file_paths)

In [6]:
unigram_inverted_index.save(r"Dumps\unigram_inverted_index.pkl")

In [7]:
stopword_list = stopwords.words('english')

def preproc(query):
    query = query.lower()
    tokenized_text = word_tokenize(query) 
    filtered_text = [word for word in tokenized_text if word not in stopword_list]
    for punc in string.punctuation:
        filtered_text = [word for word in filtered_text if word != punc]
    filtered_text = [word for word in filtered_text if word != ' ']
    return filtered_text

In [8]:
class queryHandler:
    def __init__(self, unigram_inverted_index):
        self.unigram_inverted_index = unigram_inverted_index
        self.index = unigram_inverted_index.index
        self.list_of_file_paths = unigram_inverted_index.list_of_file_paths
        self.file_names = unigram_inverted_index.file_names

    def query(self, sequence, op_sequence):
        sequence = preproc(sequence)
        op_sequence = op_sequence.split(',')
        op_sequence = [op.strip().lower() for op in op_sequence]
        doc_lists = []
        for token in sequence:
            if token in self.index:
                doc_lists.append(self.index[token])
            else:
                doc_lists.append([])
        possible_boolean_ops = ['and', 'or', 'and not', 'or not']
        for op in op_sequence:
            if op not in possible_boolean_ops:
                raise Exception(f'Invalid boolean operator {op}')
        if len(op_sequence) != len(sequence) - 1:
            raise Exception(f'Invalid query sequence. Expected {len(sequence) - 1} boolean operators, got {len(op_sequence)}')

        op_sequence_copy = op_sequence.copy()

        # Perform ops in order of precedence
        comparison_count = 0

        while op_sequence != []:
            if 'and not' in op_sequence:
                and_not_index = op_sequence.index('and not')
                op_sequence.pop(and_not_index)
                doc_lists[and_not_index], comps = self.and_not_query(doc_lists[and_not_index], doc_lists[and_not_index + 1])
                comparison_count += comps
                doc_lists.pop(and_not_index + 1)
            if 'and' in op_sequence:
                and_index = op_sequence.index('and')
                op_sequence.pop(and_index)
                doc_lists[and_index], comps = self.and_query(doc_lists[and_index], doc_lists[and_index + 1])
                comparison_count += comps
                doc_lists.pop(and_index + 1)
            if 'or not' in op_sequence:
                or_not_index = op_sequence.index('or not')
                op_sequence.pop(or_not_index)
                doc_lists[or_not_index], comps = self.or_not_query(doc_lists[or_not_index], doc_lists[or_not_index + 1])
                comparison_count += comps
                doc_lists.pop(or_not_index + 1)
            if 'or' in op_sequence:
                or_index = op_sequence.index('or')
                op_sequence.pop(or_index)
                doc_lists[or_index], comps = self.or_query(doc_lists[or_index], doc_lists[or_index + 1])
                comparison_count += comps
                doc_lists.pop(or_index + 1)

        doc_lists = sorted(doc_lists[0])

        final_query = ""

        for token, op in zip(sequence, op_sequence_copy):
            final_query += f'{token} {op.upper()} '
        final_query += sequence[-1]
        
        return {
            'docs': doc_lists,
            'comparisons': comparison_count,
            'sequence': sequence,
            'op_sequence': op_sequence_copy,
            'final_query': final_query
        }

    def not_query(self, doc_list):
        all_files = sorted(self.list_of_file_paths)
        not_docs = []
        i = 0
        j = 0
        comparisons = 0
        while i < len(all_files) and j < len(doc_list):
            comparisons += 1
            if all_files[i] == doc_list[j]:
                i += 1
                j += 1
            elif all_files[i] < doc_list[j]:
                not_docs.append(all_files[i])
                i += 1
            else:
                j += 1
        return not_docs, comparisons

    def and_query(self, doc_list1, doc_list2):
        common_docs = []
        comparisons = 0
        i = 0
        j = 0
        while i < len(doc_list1) and j < len(doc_list2):
            comparisons += 1
            if doc_list1[i] == doc_list2[j]:
                common_docs.append(doc_list1[i])
                i += 1
                j += 1
            elif doc_list1[i] < doc_list2[j]:
                i += 1
            else:
                j += 1
        return common_docs, comparisons

    def or_query(self, doc_list1, doc_list2):
        common_docs = []
        comparisons = 0
        i = 0
        j = 0
        while i < len(doc_list1) and j < len(doc_list2):
            comparisons += 1
            if doc_list1[i] == doc_list2[j]:
                common_docs.append(doc_list1[i])
                i += 1
                j += 1
            elif doc_list1[i] < doc_list2[j]:
                common_docs.append(doc_list1[i])
                i += 1
            else:
                common_docs.append(doc_list2[j])
                j += 1
        while i < len(doc_list1):
            common_docs.append(doc_list1[i])
            i += 1
        while j < len(doc_list2):
            common_docs.append(doc_list2[j])
            j += 1
        return common_docs, comparisons

    def and_not_query(self, doc_list1, doc_list2):
        not_doct_list2, not_comparisons = self.not_query(doc_list2)
        commmon_docs, and_comparisons = self.and_query(doc_list1, not_doct_list2)
        return commmon_docs, not_comparisons + and_comparisons
    
    def or_not_query(self, doc_list1, doc_list2):
        not_doct_list2, not_comparisons = self.not_query(doc_list2)
        commmon_docs, or_comparisons = self.or_query(doc_list1, not_doct_list2)
        return commmon_docs, not_comparisons + or_comparisons



In [9]:
query_handler = queryHandler(unigram_inverted_index)

In [10]:
query_handler.query('reynolds number potential', 'and, and')

{'docs': ['cranfield0530', 'cranfield0964', 'cranfield1188'],
 'comparisons': 723,
 'sequence': ['reynolds', 'number', 'potential'],
 'op_sequence': ['and', 'and'],
 'final_query': 'reynolds AND number AND potential'}

In [11]:
class mainIO:
    def __init__(self, unigram_inverted_index):
        self.unigram_inverted_index = unigram_inverted_index
        self.query_handler = queryHandler(unigram_inverted_index)

    def run(self):
        num_queries = int(input('Enter Number of Queries: '))
        ls_queries = []
        ls_op_sequences = []
        for i in range(num_queries):
            query = input('Enter Query: ')
            op_sequence = input('Enter Operator Sequence: ')
            ls_queries.append(query)
            ls_op_sequences.append(op_sequence)
        for i in range(num_queries):
            query_response = self.query_handler.query(ls_queries[i], ls_op_sequences[i])
            print(f'Query {i+1}: {query_response["final_query"]}')
            print(f'Number of documents retrieved for query {i+1}: {len(query_response["docs"])}')
            print(f'Names of the documents retrieved for query {i+1}: {query_response["docs"]}')
            print(f'Number of comparisons required for query {i+1}: {query_response["comparisons"]}')

In [12]:
mainIO(unigram_inverted_index).run()

Query 1: reynolds AND number AND potential
Number of documents retrieved for query 1: 3
Names of the documents retrieved for query 1: ['cranfield0530', 'cranfield0964', 'cranfield1188']
Number of comparisons required for query 1: 723
